# Campo de gravidade sobre a bacia do Parnaiba

Este script é baseado na disciplina de metodos potenciais. Mostra o campo de gravidade predito pelo modelo global eigen 6c4 (Förste et al., 2014). Os coeficientes da expansão em hamônicos esféricos que descreve este modelo global podem ser baixados no site do IGCEM, na página Table of models.

Förste C., Bruinsma S.L., Abrikosov O., Lemoine J.-M., Schaller T., Götze H.-J., Ebbing J., Marty J.C., Flechtner F., Balmino G., Biancale R., 2014, EIGEN-6C4 The latest combined global gravity field model including GOCE data up to degree and order 2190 of GFZ Potsdam and GRGS Toulouse, 5th GOCE User Workshop, Paris, 25-28 November 2014, url: http://icgem.gfz-potsdam.de/ICGEM/documents/Foerste-et-al-EIGEN-6C4.pdf

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from mpl_toolkits.basemap import Basemap

In [2]:
import fatiando
from fatiando import gridder

In [3]:
import minhas_funcoes as mf

### Função para plotar os estados brasileiros

In [15]:
def drawstates(ax, shapefile='BRA_adm_shp/BRA_adm0.shp'):
        shp = m.readshapefile(shapefile, 'states', drawbounds=True)
        for nshape, seg in enumerate(m.states):
            poly = Polygon(seg, facecolor='0.75', edgecolor='k')
            ax.add_patch(poly)

def drawbasin(ax, shapefile='C:/Users/flora/Google Drive/Python/EDI/shape/bacia/17.shp'):
        shp = m.readshapefile(shapefile, 'basin', drawbounds=True)
        for nshape, seg in enumerate(m.basin):
            poly = Polygon(seg, facecolor='0.75', edgecolor='k')
            ax.add_patch(poly)

### Dados baixados no site do ICGEM 

In [16]:
#Topografia predita pelo modelo ETOPO1

longitude, latitude, topografia = np.loadtxt('etopo1_eigen-6c4-131243.gdf.txt', skiprows=29, unpack=True)

topo_positiva = (topografia > 0.)
topo_negativa = (topografia < 0.)

longitude_min = np.min(longitude)
longitude_max = np.max(longitude)

latitude_min = np.min(latitude)
latitude_max = np.max(latitude)

longitude_central = 0.5*(longitude_max + longitude_min)
latitude_central = 0.5*(latitude_max + latitude_min)

area = [longitude_min, longitude_max,
        latitude_min, latitude_max]

In [17]:
area

[310.0, 320.0, -15.0, 0.0]

#### Gravidade

In [18]:
gravidade = np.loadtxt('ga_eigen-6c4-131237.txt', skiprows=35, usecols=(2,3),
                      unpack=True)

#### Distúrbio de Gravidade

In [19]:
altitude_ortometrica, disturbio = np.loadtxt('gd_eigen-6c4-131240.gdf.txt', skiprows=35, usecols=(2,3), unpack=True)

### Coordenadas x y da projeção cartográfica

A projeção foi calculada com o Basemap. Para os exemplos mostrados abaixo, a projeção escolhida foi a Transversa de Mercator.

In [20]:
m = Basemap(llcrnrlon=longitude_min,llcrnrlat=latitude_min,
            urcrnrlon=longitude_max,urcrnrlat=latitude_max,
            resolution='i',projection='tmerc',
            lon_0=longitude_central,lat_0=latitude_central)
x, y = m(longitude, latitude)

In [21]:
_ = mf.estatistica(0.001*x, 'km')

     min.:       -18.96165 km
    media:       538.34840 km
     max.:      1095.65844 km
var. max.:      1114.62009 km


In [22]:
_ = mf.estatistica(0.001*y, 'km')

     min.:        -0.00000 km
    media:       834.42908 km
     max.:      1665.08046 km
var. max.:      1665.08046 km


### Parâmetros do datum geodésico WGS84

In [23]:
a, f, GM, omega = mf.WGS84()

### Mapas

In [24]:
topografia_min, topografia_media, topografia_max, topografia_var = mf.estatistica(topografia, 'm')

     min.:     -4013.00000 m
    media:       112.71591 m
     max.:      1263.00000 m
var. max.:      5276.00000 m


In [26]:
mf.plota_mapa(m, x, y, topografia, area, 'm', 'Topografia - ETOPO1', 'gist_earth', 
              (10, 10), 2., estados=True, basin=True, escala=True, eixos=True)

#m.readshapefile('C:/Users/flora/Google Drive/Python/EDI/shape/bacia/17','17.shp',linewidth=3)

TypeError: plota_mapa() got an unexpected keyword argument 'basin'